In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict
import scipy as sp
from os.path import join as pjoin
from scipy import signal
import seaborn as sns
import scipy as sp
import os
from matplotlib.backends.backend_pdf  import PdfPages
import spirometry
import gnar
import settings

In [6]:
settings = {
    "inputfolder": r'/Users/jackdunsford/Library/CloudStorage/OneDrive-Personal/Thesis/01-220/V2 gnarpy',
    #enter the name of the file and the breath number you wish to exclude from the IC volume drift correction
    "ignoreic": [
        ["01-220_V2_IC220.txt", [12]]
    ],
    #enter the name of the file and the breath number you wish to exclude from volume drift correction and averaging
    "ignorebreath":[
        ["01-220_V2_IC220.txt", [12]]
    ],
    "saveiccorrection": False, #saves a plot with the ic trend correction to check for incorrect EELV detection
    "saverawflowvolume": False, #saves a plot of the raw flow and volume traces
    "saveflowvolumeloops": False, #saves a plot with the FV loop and MEFV for each stage
    "saveoutput": True, #saves an excel of the dataframe containing data for each stage of exercise
    "savemefv": False, #saves a figure of the MEFV curve with all FVCs
    "savemefvdata": False, #saves an excel with fvc, fev1, pef, slope ratio, and data for the MEFV curve
    "saveaveragedata": False #saves an excel with the data of the average flow volume loop for each stage
}

In [84]:
fvc_folder = r'/Users/jackdunsford/Library/CloudStorage/OneDrive-Personal/Thesis/01-102/01-102_V3/FVC'
ic_input = r'/Users/jackdunsford/Library/CloudStorage/OneDrive-Personal/Thesis/01-102/01-102_V3/ic/l-01-102_V3_IC220W.txt'
input_path = r'/Users/jackdunsford/Library/CloudStorage/OneDrive-Personal/Thesis/01-102/01-102_V3/breaths/l-01-102_V3_B220W.txt'

mefv = spirometry.mefv_curve(fvc_folder, settings)
def get_vecap(mefv, ic_input, input_path):
    with PdfPages('figure.pdf') as pdf: 
        ic = gnar.get_ic(ic_input, pdf, settings)
        avginsp_df, avgexp_df, te, ti, fb, vt, ve = gnar.averagebreaths(input_path, pdf, settings)
        erv = (spirometry.get_fvc(mefv) - ic).round(2)
        irv = (erv + vt).round(2)
        avgexp_df.volume = avgexp_df.volume + erv

    start = mefv.index[mefv['volume']==erv][0]
    end = mefv.index[mefv['volume']==irv][0]
    temax = 0
    for i in range(start, end):
        mef = mefv.flow[i]
        temax+= 0.01/mef

    ttot = te+ti
    ttotmax = temax/(te/(te+ti))
    fbmax = 60/ttotmax
    vecap = vt * fbmax


0.736558581536676
81.4599157541855
220.89892654641253
